In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select

from dateutil import parser

import googletrans

import time
from datetime import datetime

import requests
from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob

In [2]:
def url_request(url):
    driver = webdriver.Chrome()
    driver.get(url)
    return driver

In [3]:
driver= url_request('https://edition.cnn.com')

In [4]:
def keyword(word):
    cookies_button = driver.find_element(By.ID, 'onetrust-accept-btn-handler').click()
    time.sleep(3)
    search_button = driver.find_element(By.XPATH, "//button[@tabindex='-1']").click()
    search_bar  = driver.find_element(By.ID, 'header-search-bar')
    search_bar.send_keys(word)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(5)
    newest_button = driver.find_element(By.ID, 'newest').click()

    link_lst=[]
    try:
        
        
        while len(link_lst)<100:
            lnks     = driver.find_elements(By.TAG_NAME,"a")
            lst_cnn  = [lnk.get_attribute('href') for lnk in lnks]
            lst_cnn_ = [link for link in list(set(lst_cnn)) if 'https://www.cnn.com/20' in str(link)]

            for i in lst_cnn_:
                link_lst.append(i)

            list_soup=[]
            for url in link_lst:
                html = requests.get(url)
                soup = BeautifulSoup(html.content, 'html.parser')
                article = soup.find(class_='article__content')
                article = article.getText().replace('\n','').replace('      ','').replace('\xa0—\xa0','')
                list_soup.append(article)

            day_list=[]
            month_list=[]
            year_list=[]
            for url in link_lst:
                date_string = url.split("/")[3] + "-" + url.split("/")[4] + "-" + url.split("/")[5]
                dt_obj = parser.parse(date_string)

                day = dt_obj.day
                month = dt_obj.month
                year = dt_obj.year

                day_list.append(day)
                month_list.append(month)
                year_list.append(year)
            us_button = driver.find_element(By.XPATH, "//div[@class='pagination-arrow pagination-arrow-right search__pagination-link text-active']").click()

            

            dict_cnn={'article': list_soup,
                     'link':link_lst,
                     'day':day_list,
                     'month': month_list,
                     'year': year_list}
            df_cnn=pd.DataFrame(dict_cnn)
            
        return df_cnn
    
    except:
        return df_cnn


In [5]:
df = keyword('russia')

In [6]:
df

,article,link,day,month,year
0,CNN Get '5 Things' in your inbox If your d...,https://www.cnn.com/2023/03/09/us/five-things-...,9,3,2023
1,CNN House Speaker Kevin McCarthy may be t...,https://www.cnn.com/2023/03/09/politics/mccart...,9,3,2023
2,"Kyiv, UkraineCNN Russian President Vladim...",https://www.cnn.com/2023/03/09/europe/azovasta...,9,3,2023
3,CNN Protests erupted in Georgia this week...,https://www.cnn.com/2023/03/08/europe/georgia-...,8,3,2023
4,CNN The US and its allies have blocked or...,https://www.cnn.com/2023/03/09/politics/russia...,9,3,2023
5,"CNN Russia showered Kyiv, Lviv and other ...",https://www.cnn.com/2023/03/09/europe/ukraine-...,9,3,2023
6,CNN Georgia’s ruling party has withdrawn ...,https://www.cnn.com/2023/03/09/europe/georgia-...,9,3,2023
7,CNN A Russian bomber fired three hyperson...,https://www.cnn.com/2022/03/22/europe/biden-ru...,22,3,2022
8,Ukrainian President Volodymyr Zelensky joins...,https://www.cnn.com/2023/03/08/politics/zelens...,8,3,2023
9,Sign up for CNN’s Wonder Theory science newsle...,https://www.cnn.com/2023/03/08/world/permafros...,8,3,2023
